# Tutorial LightFM

MAN 3160 - Sistemas Recomendadores

En este tutorial vamos a utilizar la librería LightFM para generar máquinas de factorización para recomendación.

## Importar Librerías

In [46]:
# Instalamos librerías para descarcar y descomprimir archivos.

!pip install lightfm

In [47]:
import sys
import os

import itertools
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import lightfm
from lightfm import LightFM
from lightfm.data import Dataset
from lightfm import cross_validation

# Import LightFM's evaluation metrics
from lightfm.evaluation import precision_at_k as lightfm_prec_at_k
from lightfm.evaluation import recall_at_k as lightfm_recall_at_k

print("System version: {}".format(sys.version))
print("LightFM version: {}".format(lightfm.__version__))

System version: 3.9.0 (tags/v3.9.0:9cf6752, Oct  5 2020, 15:34:40) [MSC v.1927 64 bit (AMD64)]
LightFM version: 1.17


## Descarga y análisis de dataset

Vamos a utilizar el dataset de MovieLens. A diferencia de métodos anteriores, LightFM funciona mejor si se define la partición train/test mediante funciones de la misma librería, por lo que esta vez no vamos a utilizar una partición pre-definida y vamos a importar el dataset entero (u.data)

In [48]:
dir_train = 'ml-100k'

# Generamos los títulos de las columnas del archivo items.

columns = ['movieid', 'title', 'release_date', 'video_release_date', \
           'IMDb_URL', 'unknown', 'Action', 'Adventure', 'Animation', \
           'Children', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', \
           'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', \
           'Thriller', 'War', 'Western']

In [49]:
# Primero creamos el dataframe con los datos
df = pd.read_csv(f'{dir_train}/u.data',
                         sep='\t',
                         names=['userid', 'itemid', 'rating', 'timestamp'],
                         header=None)

In [50]:
# Cargamos el dataset con los items
df_items = pd.read_csv(f'{dir_train}/u.item',
                        sep='|',
                        index_col=0,
                        names = columns,
                        header=None,
                        encoding='latin-1')

MovieLens 100k viene también con metadata de los usuarios, como su edad y su ocupación, los cuales están disponibles en el archivo u.user

In [51]:
columns_user = ['userid', 'age', 'gender', 'occupation', 'zip_code']

In [52]:
df_users = pd.read_csv(f'{dir_train}/u.user',
                        sep='|',
                        index_col=0,
                        names = columns_user,
                        header=None,
                        encoding='latin-1')

In [53]:
df_users.head()

,age,gender,occupation,zip_code
userid,,,,
1,24,M,technician,85711
2,53,F,other,94043
3,23,M,writer,32067
4,24,M,technician,43537
5,33,F,other,15213


In [54]:
# Generamos una instancia de un objeto Dataset, la cual es la clase de LightFM para generar particiones y darle el formato
# apropiado a el set de datos

dataset = Dataset()

In [55]:
# Se ajusta el objeto dataset con la información de usuarios e ítems

dataset.fit(users=df['userid'], 
            items=df['itemid'])

# A partir de esto, podemos determinar fácilmente el número de usuarios e ítems únicos.
num_users, num_items = dataset.interactions_shape()
print(f'Num users: {num_users}, num_items: {num_items}.')

Num users: 943, num_items: 1682.


El método build_interactions(), agrega las interacciones al objeto Dataset. Este método recibe como datos el id de usuario, id de ítem y rating, los cuales, en nuestro caso corresponden a las columnas 0-3 del dataset

In [11]:
(interactions, weights) = dataset.build_interactions(df.iloc[:, 0:3].values)

La función random_train_test_split genera el split. El parámetro test_percentage=0.25 indica que el 25% de las interacciones totales quedarán en el set de testeo.

In [12]:
train_interactions, test_interactions = cross_validation.random_train_test_split(
    interactions, test_percentage=0.25)

A diferencia de otros métodos, los objetos de test y train se mantienen con una shape del tamaño |usuarios|x|items|

In [13]:
print(f"Shape of train interactions: {train_interactions.shape}")
print(f"Shape of test interactions: {test_interactions.shape}")

Shape of train interactions: (943, 1682)
Shape of test interactions: (943, 1682)


### Entrenando el modelo.

Para entrenar el modelo, lo único que se debe hacer es generar una instancia de LightFM() y usar el método .fit() para entrenarlo con los elementos que se desean. En este momento vamos a realizar un entrenamiento utilizando solo las interacciones.

In [56]:
model1 = LightFM(no_components=15, loss='logistic')

In [57]:
model1.fit(interactions=train_interactions, epochs=5)

# Agregando contenido

La librería LightFM ofrece la opción de añadir características (meta-data) tanto para los ítems como para los usuarios. De esta forma puede combinar recomendación por retroalimentación con recomendación basada en contenido.

El primer paso que debemos tomar es ajustar nuestro objeto de dataset para incluir las características que deseamos incluir.

In [58]:
all_genres = ['Action', 'Adventure', 'Animation', 'Children', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', \
              'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']

In [59]:
item_feats = []
for idx, row in df_items.iterrows():
    genres = []
    for genre in df_items.columns[5:]:
        if row[genre] == 1:
            genres.append(genre)
    item_feats.append((idx, genres))

In [18]:
item_feats[:10]

[(1, ['Animation', 'Children', 'Comedy']),
 (2, ['Action', 'Adventure', 'Thriller']),
 (3, ['Thriller']),
 (4, ['Action', 'Comedy', 'Drama']),
 (5, ['Crime', 'Drama', 'Thriller']),
 (6, ['Drama']),
 (7, ['Drama', 'Sci-Fi']),
 (8, ['Children', 'Comedy', 'Drama']),
 (9, ['Drama']),
 (10, ['Drama', 'War'])]

Hacemos lo mismo con las características de los usuarios.

In [60]:
all_occupations = sorted(list(set(df_users['occupation'])))

In [61]:
all_occupations

['administrator',
 'artist',
 'doctor',
 'educator',
 'engineer',
 'entertainment',
 'executive',
 'healthcare',
 'homemaker',
 'lawyer',
 'librarian',
 'marketing',
 'none',
 'other',
 'programmer',
 'retired',
 'salesman',
 'scientist',
 'student',
 'technician',
 'writer']

In [62]:
df_users = df_users.reset_index()

In [63]:
user_feats = [(x, [y]) for x,y in zip(df_users.userid, df_users['occupation'])]

In [64]:
user_feats[:10]

[(1, ['technician']),
 (2, ['other']),
 (3, ['writer']),
 (4, ['technician']),
 (5, ['other']),
 (6, ['executive']),
 (7, ['administrator']),
 (8, ['administrator']),
 (9, ['student']),
 (10, ['lawyer'])]

In [65]:
dataset_contenido = Dataset()
dataset_contenido.fit(users=df['userid'], 
                      items=df['itemid'], 
                      item_features=all_genres,
                      user_features=all_occupations)

In [66]:
item_features = dataset_contenido.build_item_features(item_feats)

In [67]:
user_features = dataset_contenido.build_user_features(user_feats)

In [68]:
(interactions2, weights2) = dataset_contenido.build_interactions(df.iloc[:, 0:3].values)

In [69]:
train_interactions2, test_interactions2 = cross_validation.random_train_test_split(
    interactions2, 
    test_percentage=0.25
)

In [70]:
model2 = LightFM(no_components=15, loss='logistic')
model2.fit(interactions=train_interactions2, epochs=5)

# Agregando contexto

LightFM se enfoca únicamente en agregar característica a los usuarios e ítems, por lo que no proporciona la opción de agregar información contextual que sea distinta de interacción a interacción. Sin embargo, existen algunos trucos para realizar recomendaciones basadas en contexto. Acá se muestra un método de prefiltrado contextual para los usuarios, en base a la hora en la que se realizó la calificación.

In [71]:
df

,userid,itemid,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596
...,...,...,...,...
99995,880,476,3,880175444
99996,716,204,5,879795543
99997,276,1090,1,874795795
99998,13,225,2,882399156


In [72]:
df['datetime'] = pd.to_datetime(df['timestamp'], unit='s')  # Convert the column to datetime format if it's not

# Create new columns
df['Year'] = df['datetime'].dt.year
df['Month'] = df['datetime'].dt.month
df['Weekday'] = df['datetime'].dt.weekday  # Monday=0, Sunday=6
df['Time'] = df['datetime'].dt.time
df['Hour'] = df['datetime'].dt.hour
print(df)

       userid  itemid  rating  timestamp            datetime  Year  Month  \
0         196     242       3  881250949 1997-12-04 15:55:49  1997     12   
1         186     302       3  891717742 1998-04-04 19:22:22  1998      4   
2          22     377       1  878887116 1997-11-07 07:18:36  1997     11   
3         244      51       2  880606923 1997-11-27 05:02:03  1997     11   
4         166     346       1  886397596 1998-02-02 05:33:16  1998      2   
...       ...     ...     ...        ...                 ...   ...    ...   
99995     880     476       3  880175444 1997-11-22 05:10:44  1997     11   
99996     716     204       5  879795543 1997-11-17 19:39:03  1997     11   
99997     276    1090       1  874795795 1997-09-20 22:49:55  1997      9   
99998      13     225       2  882399156 1997-12-17 22:52:36  1997     12   
99999      12     203       3  879959583 1997-11-19 17:13:03  1997     11   

       Weekday      Time  Hour  
0            3  15:55:49    15  
1        

In [73]:
def definir_bloque_horario(hour):
    if 6 <= hour < 12:
        return 'Mañana'
    elif 12 <= hour < 19:
        return 'Tarde'
    else:
        return 'Noche'

In [74]:
df['Horario'] = df['Hour'].apply(definir_bloque_horario)

In [75]:
df.head()

,userid,itemid,rating,timestamp,datetime,Year,Month,Weekday,Time,Hour,Horario
0,196,242,3,881250949,1997-12-04 15:55:49,1997,12,3,15:55:49,15,Tarde
1,186,302,3,891717742,1998-04-04 19:22:22,1998,4,5,19:22:22,19,Noche
2,22,377,1,878887116,1997-11-07 07:18:36,1997,11,4,07:18:36,7,Mañana
3,244,51,2,880606923,1997-11-27 05:02:03,1997,11,3,05:02:03,5,Noche
4,166,346,1,886397596,1998-02-02 05:33:16,1998,2,0,05:33:16,5,Noche


In [76]:
df['userid'] = df['userid'].astype(str) + '_' + df['Horario'].astype(str)

In [77]:
df.head()

,userid,itemid,rating,timestamp,datetime,Year,Month,Weekday,Time,Hour,Horario
0,196_Tarde,242,3,881250949,1997-12-04 15:55:49,1997,12,3,15:55:49,15,Tarde
1,186_Noche,302,3,891717742,1998-04-04 19:22:22,1998,4,5,19:22:22,19,Noche
2,22_Mañana,377,1,878887116,1997-11-07 07:18:36,1997,11,4,07:18:36,7,Mañana
3,244_Noche,51,2,880606923,1997-11-27 05:02:03,1997,11,3,05:02:03,5,Noche
4,166_Noche,346,1,886397596,1998-02-02 05:33:16,1998,2,0,05:33:16,5,Noche


In [78]:
df_prefiltrado = df[['userid', 'itemid', 'rating']]

In [79]:
df_prefiltrado.head()

,userid,itemid,rating
0,196_Tarde,242,3
1,186_Noche,302,3
2,22_Mañana,377,1
3,244_Noche,51,2
4,166_Noche,346,1


In [80]:
dataset_contexto = Dataset()
dataset_contexto.fit(users=df['userid'], 
                      items=df['itemid'], 
                      item_features=all_genres,
                      user_features=all_occupations)

In [81]:
df

,userid,itemid,rating,timestamp,datetime,Year,Month,Weekday,Time,Hour,Horario
0,196_Tarde,242,3,881250949,1997-12-04 15:55:49,1997,12,3,15:55:49,15,Tarde
1,186_Noche,302,3,891717742,1998-04-04 19:22:22,1998,4,5,19:22:22,19,Noche
2,22_Mañana,377,1,878887116,1997-11-07 07:18:36,1997,11,4,07:18:36,7,Mañana
3,244_Noche,51,2,880606923,1997-11-27 05:02:03,1997,11,3,05:02:03,5,Noche
4,166_Noche,346,1,886397596,1998-02-02 05:33:16,1998,2,0,05:33:16,5,Noche
...,...,...,...,...,...,...,...,...,...,...,...
99995,880_Noche,476,3,880175444,1997-11-22 05:10:44,1997,11,5,05:10:44,5,Noche
99996,716_Noche,204,5,879795543,1997-11-17 19:39:03,1997,11,0,19:39:03,19,Noche
99997,276_Noche,1090,1,874795795,1997-09-20 22:49:55,1997,9,5,22:49:55,22,Noche
99998,13_Noche,225,2,882399156,1997-12-17 22:52:36,1997,12,2,22:52:36,22,Noche


In [82]:
user_feats2 = []
horas = ['Mañana', 'Tarde', 'Noche']
usuarios = df['userid'].unique()
for uid, feats in user_feats:
    for h in horas:
        if f'{uid}_{h}' in usuarios:
            user_feats2.append((f'{uid}_{h}', feats))

In [83]:
user_feats2[:10]

[('1_Mañana', ['technician']),
 ('1_Noche', ['technician']),
 ('2_Noche', ['other']),
 ('3_Noche', ['writer']),
 ('4_Noche', ['technician']),
 ('5_Tarde', ['other']),
 ('5_Noche', ['other']),
 ('6_Noche', ['executive']),
 ('7_Tarde', ['administrator']),
 ('7_Noche', ['administrator'])]

In [84]:
item_features3 = dataset_contexto.build_item_features(item_feats)
user_features3 = dataset_contexto.build_user_features(user_feats2)
(interactions3, weights3) = dataset_contexto.build_interactions(df.iloc[:, 0:3].values)
train_interactions3, test_interactions3 = cross_validation.random_train_test_split(
    interactions3, 
    test_percentage=0.25
)

In [85]:
model3 = LightFM(no_components=15, loss='logistic')
model3.fit(interactions=train_interactions3, epochs=5)

# Evaluar los modelos

La librería LightFM tiene un modulo _evaluation_ que permite evaluar el modelo con métricas como Precision, recall, AUC y MRR de forma directa. Comparemos los resultados de los 3 modelos que entrenamos con las métricas de recall y AUC

In [86]:
from lightfm.evaluation import auc_score, recall_at_k

In [87]:
print(f'Evaluando modelo original.')
print(f'AUC score: {auc_score(model1, test_interactions, train_interactions=train_interactions).mean()}')
print(f'Recall at 20: {recall_at_k(model1, test_interactions, train_interactions=train_interactions, k=20).mean()}\n')

print(f'Evaluando modelo con contenido.')
print(f'AUC score: {auc_score(model2, test_interactions2, train_interactions=train_interactions2).mean()}')
print(f'Recall at 20: {recall_at_k(model2, test_interactions2, train_interactions=train_interactions2, k=20).mean()}\n')

print(f'Evaluando modelo con contenido y contexto.')
print(f'AUC score: {auc_score(model3, test_interactions3, train_interactions=train_interactions3).mean()}')
print(f'Recall at 20: {recall_at_k(model3, test_interactions3, train_interactions=train_interactions3, k=20).mean()}\n')

Evaluando modelo original.
AUC score: 0.8627685904502869
Recall at 20: 0.17130893508593684

Evaluando modelo con contenido.
AUC score: 0.8646020889282227
Recall at 20: 0.1713260672269536

Evaluando modelo con contenido y contexto.
AUC score: 0.8521642088890076
Recall at 20: 0.15398255041525963



### Material adicional

Si desean profundizar más y explorar ejemplos adicionales, pueden revisar la [documentación oficial de LightFM](https://making.lyst.com/lightfm/docs/index.html).

Para otros tutoriales y ejemplos, existen:
* [El cuadernillo lightfm_deep_dive proporcionado por los desarrolladores de la librería recommenders](https://github.com/recommenders-team/recommenders/blob/main/examples/02_model_hybrid/lightfm_deep_dive.ipynb)
* [Los cuadernillos de ejemplos oficiales de la librería LightFM](https://github.com/lyst/lightfm/tree/master/examples)